создать отчет из графиков и текствого описания - EDA.html

In [170]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import re



df = pd.read_csv("/home/artem/Загрузки/_data.csv")

In [171]:
pd.set_option('display.max_columns', None)

In [172]:
df.isnull().sum()
df.ffill(inplace=True)

In [173]:
def extract_rent(price_str):
    match = re.search(r'(\d+(?:\.\d+)?)\s*руб', price_str)
    if match:
        return float(match.group(1))
    return None

def extract_deposit(price_str):
    match = re.search(r'Залог\s*-\s*(\d+(?:\.\d+)?)\s*руб', price_str, re.IGNORECASE)
    if match:
        return float(match.group(1))
    return 0.0  

df['Rent'] = df['Цена'].apply(extract_rent)
df['Deposit'] = df['Цена'].apply(extract_deposit)
df['Result'] = df['Rent'] + df['Deposit']

df['Rent'] = df['Rent'].fillna(0)
df['Deposit'] = df['Deposit'].fillna(0)
df['Result'] = df['Result'].fillna(0)

df['Rent'] = df['Rent'].astype(float)
df['Deposit'] = df['Deposit'].astype(float)
df['Result'] = df['Result'].astype(float)

In [174]:
def extract_pass_lifts(lift_str): #пассажир
    if pd.isna(lift_str):
        return 0
    match = re.search(r'Пасс\s*\(?\s*(\d+)\s*\)?', lift_str, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return 0 

def extract_cargo_lifts(lift_str): #грузовые
    if pd.isna(lift_str):
        return 0
    match = re.search(r'Груз\s*\(?\s*(\d+)\s*\)?', lift_str, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return 0

df['Passenger_elevator'] = df['Лифт'].apply(extract_pass_lifts)
df['Freight_elevator'] = df['Лифт'].apply(extract_cargo_lifts)

df['Passenger_elevator'] = df['Passenger_elevator'].fillna(0).astype(int)
df['Freight_elevator'] = df['Freight_elevator'].fillna(0).astype(int)

In [175]:
def sum_room_areas(area_str):
    if pd.isna(area_str):
        return 0
    numbers = re.findall(r'\d+', area_str)
    return sum(map(int, numbers)) if numbers else 0

df['The_total_area_of_the_rooms_m2'] = df['Площадь комнат, м2'].apply(sum_room_areas)

In [176]:
df[['Number_of_floors', 'Type_of_building_of_the_house']] = df['Дом'].str.split(',', expand=True)
df['Number_of_floors'] = df['Number_of_floors'].str.strip()
df['Type_of_building_of_the_house'] = df['Type_of_building_of_the_house'].str.strip().fillna('Не указано')

In [177]:
df['Количество комнат'] = df['Количество комнат'].str.extract('(\d+)').astype(float)
среднее_количество = df['Количество комнат'].mean()
df['Количество комнат'] = df['Количество комнат'].fillna(среднее_количество)
df['Количество комнат'] = df['Количество комнат'].astype(int)

In [178]:
df[['Total_area', 'Living_area', 'Kitchen']] = df['Площадь, м2'].str.split('/', expand=True)

df['Total_area'] = df['Total_area'].fillna('0')
df['Living_area'] = df['Living_area'].fillna('0')
df['Kitchen'] = df['Kitchen'].fillna('0')

df['Total_area'] = pd.to_numeric(df['Total_area'].str.replace(',', '.'), errors='coerce').fillna(0)
df['Living_area'] = pd.to_numeric(df['Living_area'].str.replace(',', '.'), errors='coerce').fillna(0)
df['Kitchen'] = pd.to_numeric(df['Kitchen'].str.replace(',', '.'), errors='coerce').fillna(0)

In [180]:
rename_dict = {'ID  объявления': 'ID', 
               'Количество комнат': 'Number_of_rooms', 
               'Тип': 'Type', 
               'Метро': 'Metro', 
               'Адрес': 'Address', 
               'Площадь, м2': 'Area_m2', 
               'Дом': 'Home', 
               'Парковка': 'Parking', 
               'Цена': 'Price', 
               'Телефоны': 'Phone', 
               'Описание': 'Description', 
               'Ремонт': 'Renovation', 
               'Площадь комнат, м2': 'Room_area_m2', 
               'Балкон': 'Balcony', 
               'Окна': 'Windows', 
               'Санузел': 'Bathroom', 
               'Можно с детьми/животными': 'With_children/animals', 
               'Дополнительно': 'Additionally', 
               'Название ЖК': 'Name_of_the_RC', 
               'Серия дома': 'House_Series', 
               'Высота потолков, м': 'Ceiling_height_m', 
               'Лифт': 'Elevator', 
               'Мусоропровод': 'Garbage_chute', 
               'Ссылка на объявление': 'Link_to_the_ad'
              }

df = df.rename(columns=rename_dict)

,Unnamed: 0,ID,Number_of_rooms,Type,Metro,Address,Area_m2,Home,Parking,Price,Phone,Description,Renovation,Room_area_m2,Balcony,Windows,Bathroom,With_children/animals,Additionally,Name_of_the_RC,House_Series,Ceiling_height_m,Elevator,Garbage_chute,Link_to_the_ad,Rent,Deposit,Result,Passenger_elevator,Freight_elevator,The_total_area_of_the_rooms_m2,Number_of_floors,Type_of_building_of_the_house,Total_area,Living_area,Kitchen
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,500000.0,500000.0,1000000.0,4,1,0,5/16,Монолитный,200.0,20.0,0.0
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,500000.0,500000.0,1000000.0,1,1,95,5/16,Монолитно-кирпичный,198.0,95.0,18.0


In [182]:
df_cleaned = df.dropna()
df_cleaned.head(1)

,Unnamed: 0,ID,Number_of_rooms,Type,Metro,Address,Area_m2,Home,Parking,Price,Phone,Description,Renovation,Room_area_m2,Balcony,Windows,Bathroom,With_children/animals,Additionally,Name_of_the_RC,House_Series,Ceiling_height_m,Elevator,Garbage_chute,Link_to_the_ad,Rent,Deposit,Result,Passenger_elevator,Freight_elevator,The_total_area_of_the_rooms_m2,Number_of_floors,Type_of_building_of_the_house,Total_area,Living_area,Kitchen
53,53,273779074,2,Квартира,м. Смоленская (7 мин пешком),"Москва, Большой Николопесковский переулок, 3",60.0/40.0/11.0,"3/6, Кирпичный",подземная,"120000.0 руб./ За месяц, Залог - 120000 руб., ...",+79647810140,Без комиссии для арендатора. В историческом це...,Евроремонт,80+50+30+25+20,Лоджия (1),На улицу,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Ванна, Стиральная машина, К...","Филипповский, 2002",Индивидуальный проект,3.6,Пасс (1),Да,https://www.cian.ru/rent/flat/273779074,120000.0,120000.0,240000.0,1,0,205,3/6,Кирпичный,60.0,40.0,11.0


In [186]:
df.columns.tolist()

['Unnamed: 0',
 'ID',
 'Number_of_rooms',
 'Type',
 'Metro',
 'Address',
 'Area_m2',
 'Home',
 'Parking',
 'Price',
 'Phone',
 'Description',
 'Renovation',
 'Room_area_m2',
 'Balcony',
 'Windows',
 'Bathroom',
 'With_children/animals',
 'Additionally',
 'Name_of_the_RC',
 'House_Series',
 'Ceiling_height_m',
 'Elevator',
 'Garbage_chute',
 'Link_to_the_ad',
 'Rent',
 'Deposit',
 'Result',
 'Passenger_elevator',
 'Freight_elevator',
 'The_total_area_of_the_rooms_m2',
 'Number_of_floors',
 'Type_of_building_of_the_house',
 'Total_area',
 'Living_area',
 'Kitchen']